## Data Collection and Preprocessing

In [5]:
import pandas as pd
from surprise import Dataset, Reader

# Load data
movies = pd.read_csv('data/movies.csv')
ratings = pd.read_csv('data/ratings.csv')

# Preprocessing
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)


## Model Training and Evaluation

In [6]:
from surprise import SVD
from surprise.model_selection import train_test_split

# Train-test split
trainset, testset = train_test_split(data, test_size=0.25)

# Train model
algo = SVD()
algo.fit(trainset)

def get_top_n_recommendations(user_id, n=10):
    user_ratings = ratings[ratings['userId'] == user_id]
    user_movies = set(user_ratings['movieId'])
    all_movies = set(movies['movieId'])
    unseen_movies = list(all_movies - user_movies)

    predictions = [algo.predict(user_id, mid) for mid in unseen_movies]
    predictions.sort(key=lambda x: x.est, reverse=True)

    top_n_movies = [movies[movies['movieId'] == p.iid]['title'].values[0] for p in predictions[:n]]
    return top_n_movies

# Test the function
user_id = 1
top_n_movies = get_top_n_recommendations(user_id, n=10)
top_n_movies


['American Beauty (1999)',
 'Human Condition III, The (Ningen no joken III) (1961)',
 'The Jinx: The Life and Deaths of Robert Durst (2015)',
 'Usual Suspects, The (1995)',
 'Memories of Matsuko (Kiraware Matsuko no isshô) (2006)',
 'Life Is Beautiful (La Vita è bella) (1997)',
 'Duck Amuck (1953)',
 'Shawshank Redemption, The (1994)',
 '7 Plus Seven (1970)',
 'Planet Earth II (2016)']

## Flask Web Application

In [7]:
from flask import Flask, request, render_template, jsonify
import threading

app = Flask(__name__)

@app.route('/')
def index():
    return '''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Movie Recommender</title>
        <link rel="stylesheet" type="text/css" href="{{ url_for('static', filename='css/styles.css') }}">
    </head>
    <body>
        <div class="container">
            <h1>Movie Recommender</h1>
            <form id="recommendation-form">
                <label for="user_id">User ID:</label>
                <input type="text" id="user_id" name="user_id" required>
                <button type="submit">Get Recommendations</button>
            </form>
            <div id="recommendations"></div>
        </div>

        <script>
            document.getElementById('recommendation-form').addEventListener('submit', function(event) {
                event.preventDefault();
                const userId = document.getElementById('user_id').value;
                fetch(`/recommend?user_id=${userId}`)
                    .then(response => response.json())
                    .then(data => {
                        const recommendationsDiv = document.getElementById('recommendations');
                        recommendationsDiv.innerHTML = '<h2>Recommended Movies:</h2><ul>';
                        data.recommendations.forEach(movie => {
                            recommendationsDiv.innerHTML += `<li>${movie}</li>`;
                        });
                        recommendationsDiv.innerHTML += '</ul>';
                    });
            });
        </script>
    </body>
    </html>
    '''

@app.route('/recommend', methods=['GET'])
def recommend():
    user_id = int(request.args.get('user_id'))
    top_n_movies = get_top_n_recommendations(user_id)
    return jsonify({'recommendations': top_n_movies})

def run_app():
    app.run()

# Start the Flask app in a separate thread
thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
